In [55]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D , MaxPooling2D , GlobalAveragePooling2D ,Dense , Dropout
from keras.layers.merge import concatenate
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras.optimizers import Adam , RMSprop , SGD

In [21]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [72]:
def Vgg_block(layer_in ,n_fileter , n_conv ):
    for n in range(n_conv):
        layer_in =Conv2D(n_fileter , (3,3) , padding = 'same', activation = 'relu')(layer_in)
        #max pooling layers
    layer_in = MaxPooling2D((2,2) , strides = (2,2))(layer_in)
    layer_in = Dropout(0.25)(layer_in)
    return layer_in

In [74]:
#model input
input_shape = Input(shape = (28,28,1))

#add layers
layer = Vgg_block(input_shape ,64 ,2)
layer = Vgg_block(layer , 128 ,2 )
layer = Vgg_block(layer , 256 , 4)

#create model
model = Model(inputs = input_shape , outputs = layer)


In [75]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 28, 28, 64)        640       
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 28, 28, 64)        36928     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 14, 14, 128)       73856     
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 14, 14, 128)       147584    
__________

In [76]:
x = (model.output)
x = GlobalAveragePooling2D()(x)
prediction_layer = Dense(units=20,activation='softmax')(x)

In [77]:
model=Model(inputs=model.input,outputs=prediction_layer)

In [78]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 28, 28, 64)        640       
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 28, 28, 64)        36928     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 14, 14, 128)       73856     
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 14, 14, 128)       147584    
__________

In [79]:
r = 28
c = 28
dirs_train = "C:/Users/c3000094/Desktop/DataSet/Train_data/"
dirs_val =   "C:/Users/c3000094/Desktop/DataSet/Validation_data/"

In [80]:
data_gen_train = ImageDataGenerator(rescale = 1/255.)
data_gen_val = ImageDataGenerator(rescale = 1/255.)

In [89]:
train_generator = data_gen_train.flow_from_directory(dirs_train,target_size = (r,c),batch_size = 30 ,
                                                     class_mode='categorical',shuffle=True , color_mode ='grayscale')
valid_generator = data_gen_val.flow_from_directory(dirs_val , target_size=(r,c),
                                                   batch_size = 30 , class_mode = 'categorical',shuffle=True, color_mode ='grayscale')

Found 5891 images belonging to 20 classes.
Found 868 images belonging to 20 classes.


In [90]:
file_path = 'C:/Users/c3000094/Desktop/DataSet/weight_model/model_weight_vgg.h5'

In [91]:
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=15)
callbacks_list = [checkpoint, early] #early

In [92]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [93]:
step_size_train=train_generator.n//train_generator.batch_size
step_size_validation=valid_generator.n//valid_generator.batch_size
model.fit_generator(train_generator,epochs=25 , callbacks=callbacks_list , steps_per_epoch = step_size_train
                   , validation_data = valid_generator , validation_steps = step_size_validation )

Epoch 1/25
196/196 [==============================] - 69s 351ms/step - loss: 2.7257 - acc: 0.1321 - val_loss: 5.0177 - val_acc: 0.1238

Epoch 00001: val_loss improved from inf to 5.01774, saving model to C:/Users/c3000094/Desktop/DataSet/weight_model/model_weight_vgg.h5
Epoch 2/25
196/196 [==============================] - 68s 345ms/step - loss: 0.9897 - acc: 0.6804 - val_loss: 8.6756 - val_acc: 0.1360

Epoch 00002: val_loss did not improve from 5.01774
Epoch 3/25
196/196 [==============================] - 68s 346ms/step - loss: 0.2736 - acc: 0.9159 - val_loss: 10.1406 - val_acc: 0.1301

Epoch 00003: val_loss did not improve from 5.01774
Epoch 4/25
196/196 [==============================] - 67s 341ms/step - loss: 0.1293 - acc: 0.9639 - val_loss: 10.6699 - val_acc: 0.1217

Epoch 00004: val_loss did not improve from 5.01774
Epoch 5/25
196/196 [==============================] - 66s 339ms/step - loss: 0.0899 - acc: 0.9749 - val_loss: 11.6357 - val_acc: 0.1337

Epoch 00005: val_loss did not

KeyboardInterrupt: 